In [3]:
import torch
import soundfile as sf
from diffusers import StableAudioPipeline

repo_id = "stabilityai/stable-audio-open-1.0"

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

pipe = StableAudioPipeline.from_pretrained(repo_id, torch_dtype=dtype).to(device)

prompt = (
    "isolated single bass drum kick note, "
    "dry sound, clean attack, no vibrato, no reverb, no rhythm"
)

negative_prompt = "low quality, melody, rhythm, drums, reverb"

generator = torch.Generator(device).manual_seed(0) if device == "cuda" else None

audio = pipe(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=150,
    guidance_scale=6.0,
    audio_end_in_s=2.0,              # ✅ 길이 조절은 이걸로!
    num_waveforms_per_prompt=3,      # 여러 개 뽑아서 고르기
    generator=generator,
).audios

# audios는 보통 (batch, channels, samples) 형태라 저장용으로 transpose
wav = audio[0].T.float().cpu().numpy()
sf.write("stable_contrabass_note.wav", wav, pipe.vae.sampling_rate)  # 보통 44100Hz :contentReference[oaicite:1]{index=1}


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]